<a href="https://colab.research.google.com/github/JHyunjun/Attention/blob/main/Tabnet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch numpy scikit-learn pytorch-tabnet

In [ ]:
import numpy as np
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883


def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

def origin_minmax(data) : # HJ STYLE
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])

  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max_train, arr_min_train = origin_minmax(xy_train)
arr_max_test, arr_min_test = origin_minmax(xy_test)
arr_max_total = arr_max_train
arr_min_total = arr_min_train
print(arr_max_train, arr_min_train)
print(arr_max_test, arr_min_test)

for i in range(xy_train.shape[1]) : 
  print(i)
  if(arr_max_train[i] > arr_max_test[i]) : 
    arr_max_total[i] = arr_max_train[i]
  else : 
    arr_max_total[i] = arr_max_test[i]
  pass

for i in range(xy_train.shape[1]) : 
  if(arr_min_train[i] < arr_min_test[i]) : 
    arr_min_total[i] = arr_min_train[i]
  else : 
    arr_min_total[i] = arr_min_test[i]
  pass

print(arr_max_total, arr_min_total)

def HJ_MinMaxScaler(data) : 
  for i in range(data.shape[1]) : 
    numerator = data[:,i] - arr_min_total[i]
    denominator = arr_max_total[i] - arr_min_total[i]
    data[:,i] = numerator / (denominator + 1e-7)
    
  return data

def HJ_backMinMax(data) : 
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (arr_max_total[i] - arr_min_total[i]) + arr_min_total[i]

  return data

xy_train_scale = HJ_MinMaxScaler(xy_train)
xy_test_scale = HJ_MinMaxScaler(xy_test)

# Data Sequence Setting
TIME_STEPS = 5
seq_length = TIME_STEPS

train_set = xy_train_scale
test_set = xy_test_scale

def build_dataset(time_series, seq_length) : 
  dataX = []
  dataY = []
  for i in range(0, len(time_series) -seq_length) : 
    x = time_series[i:i+seq_length , : :]
    y = time_series[i + seq_length -1, :]
    dataX.append(x)
    dataY.append(y)

  return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

print(trainX.shape, trainY.shape) #(# of datas, Time_steps, features)

INPUT_DIM = trainX.shape[2]

In [ ]:
print(trainX.shape,trainY.shape)
print(testX.shape,testY.shape)

In [ ]:
time_steps = TIME_STEPS
n_features = INPUT_DIM

X_train = trainX
y_train = trainX

X_val = testX
y_val = testX

# 입력 및 출력 크기 정의
input_dim = time_steps * n_features
output_dim = time_steps * n_features

# TabNet AutoEncoder 모델 정의
tabnet_autoencoder = TabNetRegressor(n_d=16, n_a=16, n_steps=5*2, gamma=1.3, n_independent=2, n_shared=2, epsilon=1e-15,
                                     optimizer_fn=torch.optim.Adam,
                                     optimizer_params=dict(lr=0.002), scheduler_params=None, scheduler_fn=None,
                                     mask_type='entmax', input_dim=input_dim, output_dim=output_dim)

# 모델 훈련
tabnet_autoencoder.fit(X_train=X_train.reshape(-1, input_dim), y_train=X_train.reshape(-1, input_dim),
                       eval_set=[(X_val.reshape(-1, input_dim), X_val.reshape(-1, input_dim))],
                       max_epochs=100, patience=3, batch_size=32, virtual_batch_size=32)

In [ ]:
# Model Evaluation
X_val_encoded = tabnet_autoencoder.predict(X_val.reshape(-1, input_dim))
X_val_encoded = X_val_encoded.reshape(-1, time_steps, n_features)
Error = X_val - X_val_encoded

print("X_val : ",X_val.shape, X_val)
print("X_val_encoded : ",X_val_encoded.shape, X_val_encoded)
print("Error : ", Error)

testa=X_train.reshape(-1, input_dim)

In [ ]:
import matplotlib.pyplot as plt
X_train_encoded = tabnet_autoencoder.predict(X_train.reshape(-1, input_dim)).reshape(-1, time_steps, n_features)
fig, axs = plt.subplots(n_features, 2, figsize=(15, 4 * n_features))
for i in range(n_features):
    # Train 데이터
    axs[i, 0].plot(X_train[:, :, i].flatten(), label='Original')
    axs[i, 0].plot(X_train_encoded[:, :, i].flatten(), label='Reconstructed')
    axs[i, 0].set_title(f'Feature {i+1} - Train Data')
    axs[i, 0].legend()

    # Validation 데이터
    axs[i, 1].plot(X_val[:, :, i].flatten(), label='Original')
    axs[i, 1].plot(X_val_encoded[:, :, i].flatten(), label='Reconstructed')
    axs[i, 1].set_title(f'Feature {i+1} - Validation Data')
    axs[i, 1].legend()

plt.show()

#Current, Temperature, SOC, Current, Voltage순